In [5]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
class GithubClient:
    def __init__(self,token):
        self.base_url="https://api.github.com/"
        self.token = token

    def _headers(self):
        return{
            "Accept": "application/vnd.github.v3+json",
        }
    
    def _codesearch_headers(self):
        return{
            "Authorization": f"Token {self.token}",
            "Accept": "application/vnd.github.v3+json",
            "X-GitHub-Api-Version": "2022-11-28"
        }
    def code_search(self):
        url=f"{self.base_url}search/code?q=filename:package-lock.json+repo:spec-nith/Electrothon_7.0"
        response=httpx.get(url, headers=self._codesearch_headers())
        response.raise_for_status()
        return response.json()
        

    def get_repo_dependency_url(self,owner,repo):
        url = f"{self.base_url}repos/{owner}/{repo}/contents/requirements.txt"
        response = httpx.get(url, headers=self._headers())
        print(response)
        url=response.json()['download_url']
        ans=httpx.get(url)
        if response.status_code == 200:
            return ans.text
        else:
            return {"error": response.status_code, "message": response.text}
    
    def get_proj_languages(self,owner,repo):
        response=httpx.get(f"{self.base_url}repos/{owner}/{repo}/languages", headers=self._headers())
        response.raise_for_status()
        return response.json()
    
    def get_download_url(self,url):
        response=httpx.get(url)
        response_js=response.json()
        dependency_url=response_js['download_url']
        return dependency_url
    
    def dependencies_download_url(self,url):
        response=httpx.get(url)
        response_js=response.json()
        dependencies=response_js['packages']
        dict_dep=dependencies['']
        return dict_dep['dependencies']
    


In [73]:
Github_token=os.getenv("GITHUB_TOKEN")


In [74]:
a = GithubClient(Github_token)
# result = a.get_repo_dependency_url('spec-nith', 'Electrothon_7.0')
# language=a.get_proj_languages('Ashish0243', 'Customer_Service')
# print(language)
result=a.code_search()
dependecy_dict=result['items'][0]
url=dependecy_dict['url']
download_url=a.get_download_url(url)
req_dep=a.dependencies_download_url(download_url)
print(req_dep)

{'@emailjs/browser': '^4.4.1', '@fortawesome/fontawesome-svg-core': '^6.7.2', '@fortawesome/free-brands-svg-icons': '^6.7.1', '@fortawesome/free-regular-svg-icons': '^6.7.2', '@fortawesome/free-solid-svg-icons': '^6.7.2', '@fortawesome/react-fontawesome': '^0.2.2', '@lottiefiles/dotlottie-react': '^0.12.1', '@material-tailwind/react': '^2.1.10', 'autoprefixer': '^10.4.20', 'countup': '^1.8.2', 'emailjs-com': '^3.2.0', 'eslint': '^8', 'eslint-config-next': '15.0.4', 'fontawesome': '^5.6.3', 'fortawesome': '^0.0.1-security', 'framer-motion': '^11.18.2', 'fs-extra': '^11.3.0', 'gsap': '^3.12.7', 'next': '^15.5.4', 'nvm': '^0.0.4', 'postcss': '^8.4.49', 'react': '^18.3.1', 'react-dom': '^18.3.1', 'react-player': '^2.16.0', 'react-slick': '^0.30.2', 'run': '^1.5.0', 'slick-carousel': '^1.8.1', 'swiper': '^11.2.5', 'tailwind-scrollbar-hide': '^2.0.0', 'tailwindcss': '^3.4.16'}


In [2]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
load_dotenv()
api_key=os.getenv('GEMINI_API_KEY')

In [48]:
from pydantic import BaseModel,Field
from typing import List

In [3]:
model=ChatGoogleGenerativeAI(
    api_key=api_key,
    model='gemini-2.5-flash'
)

In [49]:
class Dependencies(BaseModel):
    dependencies: List[str]

structured=model.with_structured_output(Dependencies)

In [67]:
prompt=f"""You are an expert software dependency analyst specializing in Node.js and front-end frameworks.
I will provide a dictionary of dependencies from a package.json file.
Your task is to identify and strictly return only four main or top-level frameworks/libraries that a developer would have explicitly installed for building the application, and exclude all supporting or auto-installed libraries such as icons, linters, animation utilities, or styling tools.
Your filtering rules:
✅ Keep only the core, high-level frameworks/libraries such as:
   1) Next.js, React, Angular, Vue, Express, NestJS, Svelte, etc.
   2) Core UI frameworks like Tailwind CSS or Material UI.
   3) Major animation or rendering frameworks like GSAP, Framer Motion, or Three.js.
❌ Exclude:
   1) Any library starting with @, eslint, postcss, autoprefixer, or similar tooling.
   2) Minor or helper utilities like countup, emailjs-com, fs-extra, run, slick-carousel, etc.
   3) Icon packs, linters, plugins, or build tools.
Only return libraries that are fundamental to the app’s structure, rendering, or styling.
Output a clean JSON array with the most important dependency names only.
Also strictly ensure you do not repeat any similar dependency .
List of dependencies:{req_dep}
"""

In [68]:
result=structured.invoke(prompt)
print(result)

dependencies=['Next.js', 'React', 'Tailwind CSS', 'Material UI']
